CSP.01
	Variablen V:{ # Eine Variable für jede Kombination von Eigenschaft und Hausnummer
		Hausfarbe(1), Nationalität(1), Haustier(1), Getränk(1), Zigarettenmarke(1),
		Hausfarbe(2), [..]	
	}
	Domänen D:{ # X ist Platzhalter für die Hausnummern 1-5
		Hausfarbe(x) : {rot, grün, weiß, gelb, blau}
		Nationalität(x) : {England, Spanien, Ukraine, Norwegen, Japan}
		Haustier(x) : {Hund, Schnecke, Fuchs, Pferd, Zebra}
		Getränk(x) : {Kaffee, Tee, Milch, Orangensaft, Wasser}
		Zigarettenmarke(x) : {Old-Gold, Kools, Chesterfield, Lucky-Strike, Parliaments}
	}
	Constraints: {
		Keine zwei Häuser dürfen dieselbe Hausfarbe, Nationalität, Haustiere, Getränk oder Zigarettenmarke haben.
		Nationalität(x) = England <-> Hausfarbe(x) = Rot
		Nationalität(x) = Spanien <-> Haustier(x) = Hund
		Getränk(x) = Kaffee <-> Hausfarbe(x) = Grün
		Nationalität(x) = Ukraine <-> Getränk(x) = Tee
		Hausfarbe(x) = Weiss <-> Hausfarbe(x+1) = Grün		# X ist Hausnummer von links nach rechts, x+1 also das Haus rechts von x
		Zigarettenmarke = Old-Gold <-> Haustier = Schnecken
		Zigarettenmarke = Kools <-> Hausfarbe = Gelb
		Getränk(3) = Milch
		Nationalität(1) = Norwegen
		Zigarettenmarke(x) = Chesterfield <-> Haustier(x+1) = Fuchs OR Haustier(x-1) = Fuchs
		Zigarettenmarke(x) = Kools <-> Haustier(x+1) = Pferd OR Haustier(x-1) = Pferd
		Zigarettenmarke(x) = Lucky-Strike <-> Getränk(x) = Orangensaft
		Nationalität(x) = Japan <-> Zigarettenmarke(x) = Parliaments
		Nationalität(x) = Norwegen <-> Hausfarbe(x+1) = Blau OR Hausfarbe(x-1) = Blau
	}

In [ ]:
class CSP:
    def __init__(self, domains, constraints):
        self.domains = domains
        self.constraints = constraints
        self.variables = list(domains.keys()) # Tuple of House No. and Color/Drink/etc.

# Check if value assignment works with constraints
def consistent(csp, var, value, assignment):
    for other, other_val in assignment.items():
        if not csp.constraints(var, value, other, other_val):
            return False
    return True

# When testing values for variables, check if other variables have a non-empty domain
def forward_check(csp, var, value, assignment):
    removed = []

    for variables in csp.variables:
        if variables not in assignment:
            for v in list(csp.domains[variables]):

                if not csp.constraints(var, value, variables, v):
                    csp.domains[variables].remove(v)
                    removed.append((variables, v))

                if len(csp.domains[variables]) == 0:
                    for n, val in removed:
                        csp.domains[n].add(val)
                    return False, removed
    return True, removed

# Restore domains after backtrack
def restore_domains(csp, removed):
    for var, val in removed:
        csp.domains[var].add(val)

# Returns first variable without assigned value
def select_unassigned_variable(csp, assignment):
    for var in csp.variables:
        if var not in assignment:
            return var
    return None

# Backtracking search
def BT_Search(csp, assignment=None):
    if assignment is None:
        assignment = {}

    # CSP solved?
    if len(assignment) == len(csp.variables):
        return assignment

    # Choose variable to try
    var = select_unassigned_variable(csp, assignment)
    # Try assigning values
    for value in list(csp.domains[var]):
        if consistent(csp, var, value, assignment):

            assignment[var] = value

            # Check if CSP still solvable
            ok, removed = forward_check(csp, var, value, assignment)
            if ok:
                result = BT_Search(csp, assignment)
                if result is not None:
                    return result

            # Backtrack if necessary
            del assignment[var]
            restore_domains(csp, removed)

    return None

### CSP Definition
categories = ['Farbe', 'Nationalität', 'Haustier', 'Getränk', 'Zigaretten']
category_values = {
    'Farbe': ['Rot', 'Grün', 'Weiß', 'Gelb', 'Blau'],
    'Nationalität': ['England', 'Spanien', 'Ukraine', 'Norwegen', 'Japan'],
    'Haustier': ['Hund', 'Schnecken', 'Fuchs', 'Pferd', 'Zebra'],
    'Getränk': ['Kafee', 'Tee', 'Milch', 'Orangensaft', 'Wasser'],
    'Zigaretten': ['Old-Gold', 'Kools', 'Chesterfield', 'Lucky-Strike', 'Parliaments']
}
# Generate variables and their domains based on categories
variables = [(i, cat) for i in range(5) for cat in categories]
domains = {(i, cat): set(category_values[cat]) for i in range(5) for cat in categories}

def constraints(x, vx, y, vy):
    # Different colors for diff houses
    if x[0] != y[0] and vx == vy:
        return False
    return True

# CSP erstellen
csp = CSP(domains, constraints)

# Backtracking Search starten
solution = BT_Search(csp)
print(solution)


{(0, 'Farbe'): 'Grün', (0, 'Nationalität'): 'Norwegen', (0, 'Haustier'): 'Zebra', (0, 'Getränk'): 'Kafee', (0, 'Zigaretten'): 'Kools', (1, 'Farbe'): 'Blau', (1, 'Nationalität'): 'Spanien', (1, 'Haustier'): 'Hund', (1, 'Getränk'): 'Orangensaft', (1, 'Zigaretten'): 'Parliaments', (2, 'Farbe'): 'Rot', (2, 'Nationalität'): 'England', (2, 'Haustier'): 'Fuchs', (2, 'Getränk'): 'Wasser', (2, 'Zigaretten'): 'Lucky-Strike', (3, 'Farbe'): 'Gelb', (3, 'Nationalität'): 'Ukraine', (3, 'Haustier'): 'Schnecken', (3, 'Getränk'): 'Tee', (3, 'Zigaretten'): 'Old-Gold', (4, 'Farbe'): 'Weiß', (4, 'Nationalität'): 'Japan', (4, 'Haustier'): 'Pferd', (4, 'Getränk'): 'Milch', (4, 'Zigaretten'): 'Chesterfield'}
